# Short demo of using `ipython_memory_usage` to diagnose numpy and Pandas RAM usage

Author Ian uses this tool in his Higher Performance Python training (https://ianozsvald.com/training/) and it is mentioned in his High Performance Python (2nd ed, O'Reilly) technical book.

We can use it to understand how much RAM we're currently using and which of several alternate ways to solve a problem in complex tools might be the most RAM efficient solutions.

* `total RAM usage` is the current RAM usage at the end of that cell's execution
* `used` shows the difference between the _last_ `total RAM usage` and this one
* `peaked` shows any during-execution peak _above_ the resulting `total RAM usage` (i.e. hidden RAM usage that might catch you out)

In [1]:
import ipython_memory_usage
help(ipython_memory_usage) # or ipython_memory_usage?

Help on package ipython_memory_usage:

NAME
    ipython_memory_usage - Profile mem usage envelope of IPython commands and report interactively

DESCRIPTION
    Use 
    In[] %load_ext ipython_memory_usage
    In[] %imu_start # invoke magic-based tracking and
    # %imu_stop to disable

PACKAGE CONTENTS
    ipython_memory_usage
    ipython_memory_usage_perf
    perf_process

SUBMODULES
    imu

CLASSES
    IPython.core.magic.Magics(traitlets.config.configurable.Configurable)
        IPythonMemoryUsageMagics
    
    class IPythonMemoryUsageMagics(IPython.core.magic.Magics)
     |  IPythonMemoryUsageMagics(shell=None, **kwargs)
     |  
     |  # The class MUST call this class decorator at creation time
     |  # https://ipython.readthedocs.io/en/stable/config/custommagics.html
     |  
     |  Method resolution order:
     |      IPythonMemoryUsageMagics
     |      IPython.core.magic.Magics
     |      traitlets.config.configurable.Configurable
     |      traitlets.traitlets.HasTraits

In [2]:
%load_ext ipython_memory_usage
%imu_start

Enabling IPython Memory Usage, use %imu_start to begin, %imu_stop to end


'IPython Memory Usage started'

In [2] used 0.9 MiB RAM in 0.13s (system mean cpu 0%, single max cpu 0%), peaked 0.0 MiB above final usage, current RAM usage now 66.1 MiB


# Importing packages uses some RAM

In [3]:
import numpy as np # note that importing a package will increase total RAM usage a little

In [3] used 15.3 MiB RAM in 0.19s (system mean cpu 0%, single max cpu 0%), peaked 0.0 MiB above final usage, current RAM usage now 81.4 MiB


In [4]:
import pandas as pd # note that importing Pandas uses more RAM than importing numpy

In [4] used 54.1 MiB RAM in 0.39s (system mean cpu 8%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 135.5 MiB


In [5]:
import string

In [5] used 0.0 MiB RAM in 0.10s (system mean cpu 7%, single max cpu 27%), peaked 0.0 MiB above final usage, current RAM usage now 135.5 MiB


# Making a large array uses a predictable amount of RAM

In [6]:
# if we make a big array - 100M items * 8 byte floats, this cell
# uses circa 800MB (often 760 MiB - note mibi-bytes as used in the underlying memory_profiler tool)
# The total RAM usage grows by roughly this amount
arr = np.ones(100_000_000) 

In [6] used 763.2 MiB RAM in 0.28s (system mean cpu 8%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 898.7 MiB


In [7]:
# deleting arr reduces RAM usage by roughly the expected amount and
# total RAM usage should drop back down
del arr

In [7] used -762.9 MiB RAM in 0.10s (system mean cpu 8%, single max cpu 20%), peaked 762.9 MiB above final usage, current RAM usage now 135.7 MiB


In [8]:
# if we make it again, RAM usage goes up again
arr = np.ones(100_000_000) 

In [8] used 763.0 MiB RAM in 0.27s (system mean cpu 8%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 898.8 MiB


In [9]:
del arr

In [9] used -762.9 MiB RAM in 0.10s (system mean cpu 5%, single max cpu 20%), peaked 762.9 MiB above final usage, current RAM usage now 135.8 MiB


# Making a big random array takes RAM + time

In [10]:
# creating random items takes some time, after "used ... RAM" note "3s" or so for several seconds
arr = np.random.normal(size=100_000_000)
print(arr[:5], arr.dtype)

[ 1.11397493 -0.19617328  1.41230994  0.72710602  0.6042617 ] float64
In [10] used 763.1 MiB RAM in 3.25s (system mean cpu 10%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 898.9 MiB


# Intermediate calculations can cost additional temporary RAM 

**NOTE** this section may work different if you're on Windows (if so - please report back to Ian by raising a bug and noting the difference.

On some platforms, e.g. Linux as used here, temporary intermediates can be reused in-place reducing the overall memory allocation: https://docs.scipy.org/doc/numpy-1.13.0/release.html#highlights

In [11]:
pass

In [11] used 0.1 MiB RAM in 0.10s (system mean cpu 5%, single max cpu 40%), peaked 0.0 MiB above final usage, current RAM usage now 899.0 MiB


In [12]:
# (arr * 2) will allocate a new 762MiB array
# (arr * 3) will also allocate another 762MiB (so +1.4GB in total)
# the (arr * 2) array can be overwritten in-place for the division, so 
# a third temporary is _not_ needed
# the final result (costing 762MiB) is assigned to arr_result
# therefore we report "used 762MiB" at the end of the cell's execution
# plus "peaked 762MiB above final usage" due to the second temporary,
# so 1.4GB max was used during execution.
arr_result = (arr * 2) / (arr * 3)

In [12] used 763.6 MiB RAM in 0.76s (system mean cpu 10%, single max cpu 100%), peaked 762.9 MiB above final usage, current RAM usage now 1662.6 MiB


In [13]:
del arr

In [13] used -762.9 MiB RAM in 0.11s (system mean cpu 9%, single max cpu 27%), peaked 762.9 MiB above final usage, current RAM usage now 899.7 MiB


In [14]:
del arr_result

In [14] used -762.9 MiB RAM in 0.11s (system mean cpu 6%, single max cpu 30%), peaked 762.9 MiB above final usage, current RAM usage now 136.8 MiB


# Pandas DataFrames can be costly on RAM

## Example with deleting columns

Props to Jamie Brunning for this example

In [15]:
pass

In [15] used 0.0 MiB RAM in 0.10s (system mean cpu 8%, single max cpu 36%), peaked 0.0 MiB above final usage, current RAM usage now 136.8 MiB


In [16]:
arr_several_cols = np.random.normal(size=(100_000_000, 4))

In [16] used 3051.8 MiB RAM in 12.30s (system mean cpu 9%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 3188.6 MiB


In [17]:
arr_several_cols.shape

(100000000, 4)

In [17] used 0.0 MiB RAM in 0.10s (system mean cpu 7%, single max cpu 44%), peaked 0.0 MiB above final usage, current RAM usage now 3188.6 MiB


In [18]:
f"Cost per column {int(arr_several_cols.data.nbytes / arr_several_cols.shape[1]):,} bytes"

'Cost per column 800,000,000 bytes'

In [18] used 0.0 MiB RAM in 0.10s (system mean cpu 10%, single max cpu 50%), peaked 0.0 MiB above final usage, current RAM usage now 3188.6 MiB


In [19]:
# The DataFrame in this case is a thin wrapper over the numpy array
# and costs little extra RAM
df = pd.DataFrame(arr_several_cols, columns=list(string.ascii_lowercase)[:arr_several_cols.shape[1]])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000000 entries, 0 to 99999999
Data columns (total 4 columns):
 #   Column  Dtype  
---  ------  -----  
 0   a       float64
 1   b       float64
 2   c       float64
 3   d       float64
dtypes: float64(4)
memory usage: 3.0 GB
In [19] used 2.1 MiB RAM in 0.11s (system mean cpu 8%, single max cpu 33%), peaked 0.0 MiB above final usage, current RAM usage now 3190.7 MiB


In [20]:
# use Jupyter's xdel to remove all references of our expensive array, just in case
# (but not in this case) it is also referred to in an Out[] history item
%xdel arr_several_cols

In [20] used 0.0 MiB RAM in 0.10s (system mean cpu 9%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 3190.7 MiB


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000000 entries, 0 to 99999999
Data columns (total 4 columns):
 #   Column  Dtype  
---  ------  -----  
 0   a       float64
 1   b       float64
 2   c       float64
 3   d       float64
dtypes: float64(4)
memory usage: 3.0 GB
In [21] used 0.0 MiB RAM in 0.11s (system mean cpu 11%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 3190.7 MiB


In [22]:
# deleting a column 
# note that no RAM is freed up!
del df['a']

In [22] used 0.0 MiB RAM in 0.10s (system mean cpu 9%, single max cpu 30%), peaked 0.0 MiB above final usage, current RAM usage now 3190.7 MiB


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000000 entries, 0 to 99999999
Data columns (total 3 columns):
 #   Column  Dtype  
---  ------  -----  
 0   b       float64
 1   c       float64
 2   d       float64
dtypes: float64(3)
memory usage: 2.2 GB
In [23] used 0.0 MiB RAM in 0.11s (system mean cpu 12%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 3190.7 MiB


In [24]:
# we get no benefit by forcing a collection
import gc
gc.collect()

0

In [24] used 0.0 MiB RAM in 0.14s (system mean cpu 11%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 3190.7 MiB


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000000 entries, 0 to 99999999
Data columns (total 3 columns):
 #   Column  Dtype  
---  ------  -----  
 0   b       float64
 1   c       float64
 2   d       float64
dtypes: float64(3)
memory usage: 2.2 GB
In [25] used 0.0 MiB RAM in 0.11s (system mean cpu 12%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 3190.7 MiB


In [26]:
pass

In [26] used 0.0 MiB RAM in 0.10s (system mean cpu 11%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 3190.7 MiB


In [27]:
# using drop with inplace=False (the default) returns a copied DataFrame, if you don't use
# this then maybe you end up with multiple DataFrames consuming RAM in a confusing fashion
# e.g. you might have done `df2 = df.drop...` and then you've got the unmodified original
# plus the modified df2 in the local namespace
# We see total RAM usage drop by circa 800MB, the cost of 1 column, plus the other column (a)
# maybe the usage of drop forces a flush on any internal caching in pandas?
df = df.drop(columns=['b'])

In [27] used -1525.7 MiB RAM in 1.00s (system mean cpu 17%, single max cpu 100%), peaked 2937.5 MiB above final usage, current RAM usage now 1665.0 MiB


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000000 entries, 0 to 99999999
Data columns (total 2 columns):
 #   Column  Dtype  
---  ------  -----  
 0   c       float64
 1   d       float64
dtypes: float64(2)
memory usage: 1.5 GB
In [28] used 0.0 MiB RAM in 0.11s (system mean cpu 11%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 1665.0 MiB


In [29]:
# dropping in-place is probably more sensible, we recover another circa 800MB
df.drop(columns=['c'], inplace=True)

In [29] used -762.9 MiB RAM in 0.44s (system mean cpu 14%, single max cpu 80%), peaked 762.9 MiB above final usage, current RAM usage now 902.1 MiB


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000000 entries, 0 to 99999999
Data columns (total 1 columns):
 #   Column  Dtype  
---  ------  -----  
 0   d       float64
dtypes: float64(1)
memory usage: 762.9 MB
In [30] used 0.0 MiB RAM in 0.11s (system mean cpu 0%, single max cpu 0%), peaked 0.0 MiB above final usage, current RAM usage now 902.1 MiB


In [31]:
pass

In [31] used 0.0 MiB RAM in 0.10s (system mean cpu 0%, single max cpu 0%), peaked 0.0 MiB above final usage, current RAM usage now 902.1 MiB


In [32]:
# now we get back to where we were before we made the DataFrame and the array
df.drop(columns=['d'], inplace=True)

In [32] used -762.9 MiB RAM in 0.11s (system mean cpu 0%, single max cpu 0%), peaked 0.0 MiB above final usage, current RAM usage now 139.1 MiB


# Diagnostics

`%xdel my_df` will delete all references of `my_df` from the namespace including those in the Out[] history buffer, this does more cleaning than just using `del my_df`.

`%reset` will reset all variables and imported modules, it is like starting a new kernel.

In [33]:
# %whos shows what's in the local namespace
%whos

Variable               Type         Data/Info
---------------------------------------------
df                     DataFrame    Empty DataFrame\nColumns:<...>0000000 rows x 0 columns]
gc                     module       <module 'gc' (built-in)>
ipython_memory_usage   module       <module 'ipython_memory_u<...>emory_usage/__init__.py'>
np                     module       <module 'numpy' from '/ho<...>kages/numpy/__init__.py'>
pd                     module       <module 'pandas' from '/h<...>ages/pandas/__init__.py'>
string                 module       <module 'string' from '/h<...>ib/python3.11/string.py'>
In [33] used 0.0 MiB RAM in 0.10s (system mean cpu 6%, single max cpu 22%), peaked 0.0 MiB above final usage, current RAM usage now 139.1 MiB


In [34]:
# we can use %xdel to safely remove all references including those that might be (but not in this case)
# in the Out[] history buffer
%xdel df

In [34] used 0.0 MiB RAM in 0.10s (system mean cpu 5%, single max cpu 25%), peaked 0.0 MiB above final usage, current RAM usage now 139.1 MiB


In [35]:
#%imu_stop
#'IPython Memory Usage stopped'

In [35] used 0.0 MiB RAM in 0.10s (system mean cpu 6%, single max cpu 20%), peaked 0.0 MiB above final usage, current RAM usage now 139.1 MiB
